# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [54]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [55]:
housing = pd.read_csv('cal_housing_clean.csv')
housing.shape

(20640, 7)

In [56]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [57]:
housing.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X = housing.drop('medianHouseValue', axis=1)
y = housing.medianHouseValue

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(14448, 6) (6192, 6)
(14448,) (6192,)


### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [61]:
from sklearn.preprocessing import MinMaxScaler

In [62]:
xscaler = MinMaxScaler().fit(X_train)

In [69]:
X_train_scaled = pd.DataFrame(xscaler.transform(X_train), 
                              columns=[col+'_scaled' for col in X_train.columns], 
                              index=X_train.index)
X_train_scaled.shape

(14448, 6)

In [70]:
X_test_scaled = pd.DataFrame(xscaler.transform(X_test), 
                             columns=[col+'_scaled' for col in X_test.columns], 
                             index=X_test.index)
X_test_scaled.shape

(6192, 6)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [78]:
feat_cols = [tf.feature_column.numeric_column(col) for col in X_train_scaled.columns]
feat_cols

[_NumericColumn(key='housingMedianAge_scaled', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='totalRooms_scaled', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='totalBedrooms_scaled', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='population_scaled', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='households_scaled', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='medianIncome_scaled', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [79]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train_scaled, y=y_train, 
                                                batch_size=10, num_epochs=1000, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [80]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[6,6,6], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_tf_random_seed': 1, '_session_config': None, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_model_dir': '/var/folders/n_/yc8w5lvd0939ndnjzj5wm8mr0000gn/T/tmpdf3o2s_2', '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [93]:
dnn_model.train(input_fn=input_func, steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /var/folders/n_/yc8w5lvd0939ndnjzj5wm8mr0000gn/T/tmpdf3o2s_2/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into /var/folders/n_/yc8w5lvd0939ndnjzj5wm8mr0000gn/T/tmpdf3o2s_2/model.ckpt.
INFO:tensorflow:step = 1001, loss = 3.37499e+11
INFO:tensorflow:global_step/sec: 619.613
INFO:tensorflow:step = 1101, loss = 2.30776e+11 (0.162 sec)
INFO:tensorflow:global_step/sec: 594.686
INFO:tensorflow:step = 1201, loss = 1.43816e+11 (0.168 sec)
INFO:tensorflow:global_step/sec: 640.8
INFO:tensorflow:step = 1301, loss = 1.79766e+11 (0.156 sec)
INFO:tensorflow:global_step/sec: 582.001
INFO:tensorflow:step = 1401, loss = 2.13569e+11 (0.172 sec)
INFO:tensorflow:global_step/sec: 648.013
INFO:tensorflow:step = 1501, loss = 4.3805e+11 (0.154 sec)
INFO:tensorflow:global_step/sec: 619.087
INFO:tensorflow:step = 1601, loss = 2.2981e+11 (0.162 sec)
INFO:tensorflow:global_step/sec: 602.25
INFO:tensorflow:step = 17

INFO:tensorflow:global_step/sec: 593.827
INFO:tensorflow:step = 8901, loss = 7.04869e+10 (0.168 sec)
INFO:tensorflow:global_step/sec: 474.591
INFO:tensorflow:step = 9001, loss = 7.67317e+10 (0.215 sec)
INFO:tensorflow:global_step/sec: 558.656
INFO:tensorflow:step = 9101, loss = 1.30535e+11 (0.177 sec)
INFO:tensorflow:global_step/sec: 555.25
INFO:tensorflow:step = 9201, loss = 1.48933e+11 (0.178 sec)
INFO:tensorflow:global_step/sec: 557.982
INFO:tensorflow:step = 9301, loss = 1.34846e+11 (0.180 sec)
INFO:tensorflow:global_step/sec: 500.023
INFO:tensorflow:step = 9401, loss = 5.98918e+10 (0.205 sec)
INFO:tensorflow:global_step/sec: 626.555
INFO:tensorflow:step = 9501, loss = 1.05588e+11 (0.155 sec)
INFO:tensorflow:global_step/sec: 498.582
INFO:tensorflow:step = 9601, loss = 3.62485e+10 (0.200 sec)
INFO:tensorflow:global_step/sec: 602.471
INFO:tensorflow:step = 9701, loss = 1.00388e+11 (0.165 sec)
INFO:tensorflow:global_step/sec: 598.448
INFO:tensorflow:step = 9801, loss = 2.6764e+10 (0.1

INFO:tensorflow:global_step/sec: 481.454
INFO:tensorflow:step = 25001, loss = 9.57338e+10 (0.210 sec)
INFO:tensorflow:global_step/sec: 469.226
INFO:tensorflow:step = 25101, loss = 1.06168e+11 (0.210 sec)
INFO:tensorflow:global_step/sec: 656.815
INFO:tensorflow:step = 25201, loss = 4.1844e+10 (0.152 sec)
INFO:tensorflow:global_step/sec: 609.273
INFO:tensorflow:step = 25301, loss = 1.54588e+11 (0.163 sec)
INFO:tensorflow:global_step/sec: 512.542
INFO:tensorflow:step = 25401, loss = 1.62004e+11 (0.198 sec)
INFO:tensorflow:global_step/sec: 450.035
INFO:tensorflow:step = 25501, loss = 7.36181e+10 (0.222 sec)
INFO:tensorflow:global_step/sec: 532.868
INFO:tensorflow:step = 25601, loss = 2.14772e+11 (0.189 sec)
INFO:tensorflow:global_step/sec: 449.03
INFO:tensorflow:step = 25701, loss = 3.29196e+10 (0.220 sec)
INFO:tensorflow:global_step/sec: 583.699
INFO:tensorflow:step = 25801, loss = 1.05084e+11 (0.172 sec)
INFO:tensorflow:global_step/sec: 556.069
INFO:tensorflow:step = 25901, loss = 1.0328

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [94]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test_scaled, y=y_test, 
                                                      batch_size=10, num_epochs=1, shuffle=False)

In [95]:
y_pred = dnn_model.predict(pred_input_func)
y_pred_list = [p['predictions'][0] for p in y_pred]

INFO:tensorflow:Restoring parameters from /var/folders/n_/yc8w5lvd0939ndnjzj5wm8mr0000gn/T/tmpdf3o2s_2/model.ckpt-26000


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [96]:
from sklearn.metrics import mean_squared_error

In [98]:
np.sqrt(mean_squared_error(y_test, y_pred_list))

96499.609419587068

97921.93181985477